OSIsoft Cloud Services REST API - create and write stream events

Requirements:

Juptyer notebook or lab
Python
ocs_sample_library https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/library_samples/Python3

config.ini configured for your OSIsoft Cloud Services environment https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/basic_samples/SDS/Python/SDSPy/Python3

objective:

Create a type, stream and write stream events


In [1]:
# specify a unique prefix to create objects
example_prefix = 'SDSUom.'

In [2]:
# setup the environment

import configparser
import logging
from ocs_sample_library_preview import *
# For testing/running modified github version
import json
import datetime

# setup logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# Read the configuration informatation for your OSIsoft Cloud Services acccount from config.ini

config = configparser.ConfigParser()
config.read('config.ini')

ocsClient = OCSClient(config.get('Access', 'ApiVersion'), config.get('Access', 'Tenant'), config.get('Access', 'Resource'), 
                        config.get('Credentials', 'ClientId'), config.get('Credentials', 'ClientSecret'))
        
namespace_id = config.get('Configurations', 'Namespace')

In [3]:
# Dictionary to describe information about the type

thetype = {'sdstypecode': SdsTypeCode.Object, 
           'id': f'{example_prefix}bearing_channel',
           'name': 'bearing_channel', 
           'description': 'bearing vibration signals'}

# sds type properties
timestamp_property = SdsTypeProperty()
timestamp_property.Id = "timestamp"
timestamp_property.SdsType = SdsType.fromDictionary({"SdsTypeCode": SdsTypeCode.DateTime.value})
timestamp_property.IsKey = True

index_property = SdsTypeProperty()
index_property.Id = "index"
index_property.SdsType = SdsType.fromDictionary({"SdsTypeCode": SdsTypeCode.Decimal.value})

channel_property =  SdsTypeProperty()
channel_property.Id = "channel"
channel_property.SdsType = SdsType.fromJson({"SdsTypeCode" : SdsTypeCode.Double.value })

# sds type defintion
bearing = SdsType()
bearing.Id = thetype['id']
bearing.SdsTypeCode = thetype['sdstypecode']
bearing.Name = thetype['name']
bearing.Description=thetype['description']
bearing.Properties = [timestamp_property,index_property,channel_property]

# Create the type
bearing_type = ocsClient.Types.getOrCreateType(namespace_id, bearing)
print(json.dumps(json.loads(bearing_type.toJson()),indent=4))

{
    "SdsTypeCode": 1,
    "Properties": [
        {
            "IsKey": true,
            "Id": "timestamp",
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 16,
                "Id": null,
                "Name": null,
                "Description": null
            },
            "Value": null,
            "Order": null
        },
        {
            "IsKey": false,
            "Id": "index",
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 15,
                "Id": null,
                "Name": null,
                "Description": null
            },
            "Value": null,
            "Order": null
        },
        {
            "IsKey": false,
            "Id": "channel",
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 14,
                "Id": null,
                "Name": null,
                "Description": null
            },
            "Value": n

In [4]:
# Create 4 streams based on the created type
for bearing in range (1,5):
    stream = SdsStream(id=f'{example_prefix}Bearing{bearing}',name=f'bearing{bearing}', description=f'Bearing {bearing} Channel 1', typeId=f'{example_prefix}bearing_channel')
    stream = ocsClient.Streams.getOrCreateStream(namespace_id,stream)
    print(f'id: {stream.Id}, name: {stream.Name}')

id: SDSUom.Bearing1, name: bearing1
id: SDSUom.Bearing2, name: bearing2
id: SDSUom.Bearing3, name: bearing3
id: SDSUom.Bearing4, name: bearing4


In [5]:
# create stream events
values = [ {'timestamp': '2004-02-19T06:19:00Z', 'counter': 0, 'channel': -0.872},
           {'timestamp': '2004-02-19T06:21:00Z', 'counter': 1, 'channel': 0.002},
           {'timestamp': '2004-02-19T06:23:00Z', 'counter': 2, 'channel': -0.438},
           {'timestamp': '2004-02-19T06:25:00Z', 'counter': 3, 'channel': 0.222},
           {'timestamp': '2004-02-19T06:27:00Z', 'counter': 4, 'channel': 0.678}]
# insert events
for bearing in range(1,5):
        ocsClient.Streams.insertValues(namespace_id,f'{example_prefix}Bearing{bearing}',json.dumps(values))

In [6]:
# display the stream events
for stream in ocsClient.Streams.getStreams(namespace_id,query=f'{example_prefix}Bearing*'):
    value = ocsClient.Streams.getRangeValues(namespace_id,stream.Id,start="2001-01-01",count=10,value_class=None,skip=0,reverse=False,boundary_type=SdsBoundaryType.Inside)
    print(f'{stream.Id}: {value}')


SDSUom.Bearing4: [{'timestamp': '2004-02-19T06:19:00Z', 'channel': -0.872}, {'timestamp': '2004-02-19T06:21:00Z', 'channel': 0.002}, {'timestamp': '2004-02-19T06:23:00Z', 'channel': -0.438}, {'timestamp': '2004-02-19T06:25:00Z', 'channel': 0.222}, {'timestamp': '2004-02-19T06:27:00Z', 'channel': 0.678}]
SDSUom.Bearing2: [{'timestamp': '2004-02-19T06:19:00Z', 'channel': -0.872}, {'timestamp': '2004-02-19T06:21:00Z', 'channel': 0.002}, {'timestamp': '2004-02-19T06:23:00Z', 'channel': -0.438}, {'timestamp': '2004-02-19T06:25:00Z', 'channel': 0.222}, {'timestamp': '2004-02-19T06:27:00Z', 'channel': 0.678}]
SDSUom.Bearing1: [{'timestamp': '2004-02-19T06:19:00Z', 'channel': -0.872}, {'timestamp': '2004-02-19T06:21:00Z', 'channel': 0.002}, {'timestamp': '2004-02-19T06:23:00Z', 'channel': -0.438}, {'timestamp': '2004-02-19T06:25:00Z', 'channel': 0.222}, {'timestamp': '2004-02-19T06:27:00Z', 'channel': 0.678}]
SDSUom.Bearing3: [{'timestamp': '2004-02-19T06:19:00Z', 'channel': -0.872}, {'timesta